### <b> Calculating the luminosity of a polytrope of index n </b>

In [1]:
#from polytrope_model import lane_emden_sys, lane_emden_solver
from scipy.integrate import cumulative_simpson, cumulative_trapezoid
import matplotlib.pyplot as plt
import numpy as np

from polytrope_model import *

In [2]:
# Chemical compostion (H and He)
def chemical_composition(xi, X_s, r_c, d_c, R, time, total_time):

    # Hidrogen
    X_c = X_s - X_s * (time/total_time)
    X = X_c + (X_s - X_c)/(1+np.exp((r_c-xi)/(R*d_c)))

    return X

In [3]:
# Thermodynamic functions (general and center)

def density(M, R, n):
    xi, y1, y2 = lane_emden_solver(n)
    a_n = -xi[-1]/(3*y2[-1])
    central_density = M/(4/3*np.pi*R**3) * a_n
    density = central_density * y1**n

    return density, central_density

def pressure(M, R, n):
    xi, y1, y2 = lane_emden_solver(n)
    c_n = 1 / (4 * np.pi * (n+1) * y2[-1]**2)
    central_pressure = (G*M**2)/R**4 * c_n
    pressure = central_pressure * y1**(n+1)

    return pressure, central_pressure

def temperature(M, R, X, Z, n): # R_g = gas constant, G  (define both as global variables?)
    xi, y1, y2 = lane_emden_solver(n)
    mu = 4/(5*X-Z+3) # needs to come from chemical composition array 
    
    b_n = 1 / (n+1) * xi[-1] * (-y2[-1])
    central_temperature = mu*G*M/(R_g*R) * b_n
    temperature = central_temperature *  y1

    return temperature, central_temperature

In [4]:
def emissivity(rho, T, X, Z):
    T6 = T/1e6

    T_og_length = len(T6) # original length of T6 array
    T_new_length = 1 + int(np.argwhere(T6<1e-3)[0]) # the remaining values of T6 might cause numerically unstable values of emissivity 
    size_dif = T_og_length - T_new_length

    # Update array sizes for emissivity calculations
    T6 = T6[:T_new_length]
    X = X[:T_new_length]
    rho = rho[:T_new_length]

    # Formula variables
    alpha = 1.2e17*((1-X-Z)/(4*X))**2 * np.exp(-100*T6**(-1/3))
    eps_0 = 2.38e6* X** 2 * rho* T6** (-2/3) * (1 + 0.0123*T6** (1/3) + 0.0109*T6** (2/3) + 0.00095*T6) * np.exp(-33.80*T6** (-1/3) + 0.27*rho** (1/2)* T6**(-3/2))
    phi_alpha = 1-alpha + np.sqrt(alpha*(alpha+2))
    F1 = (np.sqrt(alpha+2) - np.sqrt(alpha))/(np.sqrt(alpha+2) + 3*np.sqrt(alpha))
    F2 = (1-F1)/(1+8.94e15 * (X/(4-3*X))* T6**(-1/6) * np.exp(-102.65*T6**(-1/3)))
    F3 = 1-F1-F2

    # Emissivity: proton-proton (e_pp), carbon-nitrogen-oxigen (e_cno)
    e_pp = (eps_0/0.980) * phi_alpha * (0.980*F1 + 0.960*F2 + 0.721*F3)
    e_cno = 8.67e27*Z*X*rho*T6** (-2/3) * (1 + 0.0027*T6** (1/3) - 0.00778*T6** (2/3) - 0.000149*T6) * np.exp(-152.28*T6**(-1/3))

    # Fill the arrays back to their original sizes with zeros (further calculations using emissivity won't be affected)
    zeros = np.zeros(size_dif)
    e_pp = np.concatenate((e_pp, zeros))
    e_cno = np.concatenate((e_cno, zeros))

    # Total emissivity
    e_total = e_pp + e_cno

    return e_total, e_pp, e_cno

In [5]:
def luminosity(n, X_s, r_c, d_c, R, time, total_time, M, Z,L_sol ):

    # xi, theta(xi), theta'(xi)
    xi, y1, y2 = lane_emden_solver(n)

    # chemical composition X, Y
    X = chemical_composition(xi, X_s, r_c, d_c, R, time, total_time)

    # Thermodynamic functions (general and center)
    rho, rho_c = density(M, R, n)
    P, P_c = pressure(M, R, n)
    T, T_c = temperature(M, R, X, Z, n)

    # Emissivity
    ems_tot, e_pp, e_cno = emissivity(rho, T, X, Z)

    # Constant of the luminosity formula
    const = -1/(xi[-1]**2 * y2[-1])

    # Integrand of the luminosity formula
    integrand = xi**2 * y1**n * M * ems_tot / L_sol
    

    # Cumulative integral
    integral = cumulative_trapezoid(integrand, x=xi, initial=0)
    return const * integral


#### <b> Testing emissivity for Sun </b>

In [8]:
# Global
G = 6.67408e-8 # gravitational constant
R_g = 8.314511e7 # gas constant

#Sun
M_sol = 1.988475e33
L_sol = 3.828e33
R_sol = 6.957e10
Z_sol = 0.02857
Y_sol = 0.28
total_time = 2*4.6*M_sol*(1/L_sol)

'''
n_sol = 3.08692
X_s_sol = (1-Y_sol)/(1+Z_sol)
Z_sol = 1-Y_sol-X_s_sol
xi, y, z = lane_emden_solver(n_sol)
X_sol = chemical_composition(xi, X_s_sol, 0.2, 0.03, R_sol, 1, 2)
rho, rho_c = density(M_sol, R_sol, n_sol)
P, P_c = pressure(M_sol, R_sol, n_sol)
T, T_c = temperature(M_sol, R_sol, X_sol, Z_sol, n_sol)
e, epp, ecno = emissivity(rho, T, X_sol, Z_sol)

sun_lum = luminosity(n_sol, X_s_sol, 0.2, 0.03, R_sol, 1, 2, M_sol, Z_sol, L_sol )
'''

'\nn_sol = 3.08692\nX_s_sol = (1-Y_sol)/(1+Z_sol)\nZ_sol = 1-Y_sol-X_s_sol\nxi, y, z = lane_emden_solver(n_sol)\nX_sol = chemical_composition(xi, X_s_sol, 0.2, 0.03, R_sol, 1, 2)\nrho, rho_c = density(M_sol, R_sol, n_sol)\nP, P_c = pressure(M_sol, R_sol, n_sol)\nT, T_c = temperature(M_sol, R_sol, X_sol, Z_sol, n_sol)\ne, epp, ecno = emissivity(rho, T, X_sol, Z_sol)\n\nsun_lum = luminosity(n_sol, X_s_sol, 0.2, 0.03, R_sol, 1, 2, M_sol, Z_sol, L_sol )\n'

In [9]:
#Sol
M_sol = 1.988475e33
L_sol = 3.828e33
R_sol = 6.957e10
Z_sol = 0.02857
Y_sol = 0.28

#%%
# Epsilon Eridani
M_e = 0.82 * M_sol
dM_e = 0.02 * M_sol
R_e = 0.738 * R_sol
dR_e= 0.0003 * R_sol
L_e = 0.32 * L_sol
dL_e = 0.01 * L_sol
metal_e = -0.08
dmetal_e = 0.01
Z_e = Z_sol*10**metal_e
Y_e = 0.2423
dY_e = 0.0054
total_time_e = 2*4.6*M_e*(1/L_e)


# Theta Persei
M_t = 1.138 * M_sol
dM_t = 0.010 * M_sol
R_t = 1.319 * R_sol
dR_t = 0.011 * R_sol
L_t = 2.235 * L_sol
dL_t = 0.040 * L_sol
metal_t = -0.03
dmetal_t = 0.09
Z_t = Z_e = Z_sol*10**metal_t
Y_t = 0.2423
dY_t = 0.0054
total_time_t = 2*4.6*M_e*(1/L_t)

In [ ]:
# Indexes
poly_index_t = index(y, 2.5, 4.5, M_t, L_t, Y_t, Z_t, R_t, L_sol, 0.2, 0.03, total_time_t/3, total_time_t)
poly_index_e = index(y, 1.2, 4.5, M_e, L_e, Y_e, Z_e, R_e, L_sol, 0.2, 0.03, total_time_e/2, total_time_e)
poly_index_sun = index(y, 1.5, 4, M_sol, L_sol, Y_sol, Z_sol, R_sol,  L_sol, 0.2, 0.03, 1, 2)

# Theta
Xs_t = (1-Y_t)/(1+Z_t)
Z = 1-Y_t-Xs_t
x_t, y, z = lane_emden_solver(poly_index_t)
X_t = chemical_composition(x_t, Xs_t, 0.2, 0.03, total_time_t/2, total_time_t)

# Epsilon
Xs_e = (1-Y_e)/(1+Z_e)
Z = 1-Y_e-Xs_e
x_e, y, z = lane_emden_solver(poly_index_e)
X_e = chemical_composition(x_e, Xs_e, 0.2, 0.03, total_time_e/2, total_time_e)

# Sun
Xs_sun = (1-Y_sol)/(1+Z_sol)
Z = 1-Y_sol-Xs_sun
x_sol, y, z = lane_emden_solver(poly_index_sun)
X_sun = chemical_composition(x_sol, Xs_sun, 0.2, 0.03, 1, 2)

TypeError: 'numpy.ndarray' object is not callable

In [ ]:
# Thermodynamic
rho_sun, P_sun, T_sun = rho_P_T(poly_index_sun, R_sol, M_sol, X_sol, Z_sol)
rho_e, P_e, T_e = rho_P_T(poly_index_e, R_e, M_e, X_e, Z_e)
rho_t, P_t, T_t = rho_P_T(poly_index_t, R_t, M_t, X_t, Z_t)

def thermo_plots(xi, rho_sun, rho_e, rho_t):
    plt.plot(xi, rho_sun, color='red', label='Sun')
    plt.plot(xi, rho_e, color = 'yellow', label='Epsilon Eridani')
    plt.plot(xi, rho_t, color = 'blue', label=r'$\theta Persei A$')
    plt.show()

NameError: name 'density' is not defined

In [3]:
# Temperature

In [5]:
# Pressure

In [4]:
# Emi CNO

In [2]:
# Emi PP